# Анализ дата сета из базы самолётов

### Импортируем библиотеки, сами данные и даём заголовки колонкам

In [1]:
import pandas as pd
import numpy as np
import datetime


columns=['flight_id', 'origin', 'destination', 'duration', 'aircraft_model', 'total_passengers', 'total_seats', 'fuel_expence', 'tickets_total_cost', 'income', 'departure_date', 'price_per_km']
data = pd.read_csv('flights.csv', names=columns)


data.head()

,flight_id,origin,destination,duration,aircraft_model,total_passengers,total_seats,fuel_expence,tickets_total_cost,income,departure_date,price_per_km
0,136560,Anapa,Novokuznetsk,305,Boeing 737-300,0,130,10615.0,NaN,NaN,2017-01-24,NaN
1,136511,Anapa,Novokuznetsk,310,Boeing 737-300,0,130,21225.0,NaN,NaN,2017-02-21,NaN
2,136514,Anapa,Novokuznetsk,299,Boeing 737-300,0,130,125202.0,NaN,NaN,2017-02-07,NaN
3,136518,Anapa,Novokuznetsk,304,Boeing 737-300,0,130,8493.0,NaN,NaN,2016-12-20,NaN
4,136523,Anapa,Novokuznetsk,312,Boeing 737-300,0,130,25469.0,NaN,NaN,2017-01-10,NaN


Напомню, что колонка "price_per_km" представляет собой отношение прибыли на каждый пролетевший километр. Придуманный мной косвенный показатель прибыльности рейса. 

### Посмотрим информацию по данным

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   flight_id           193 non-null    int64  
 1   origin              193 non-null    object 
 2   destination         193 non-null    object 
 3   duration            193 non-null    int64  
 4   aircraft_model      193 non-null    object 
 5   total_passengers    193 non-null    int64  
 6   total_seats         193 non-null    int64  
 7   fuel_expence        193 non-null    float64
 8   tickets_total_cost  180 non-null    float64
 9   income              180 non-null    float64
 10  departure_date      193 non-null    object 
 11  price_per_km        180 non-null    float64
dtypes: float64(4), int64(4), object(4)
memory usage: 18.2+ KB


In [3]:
data.describe()

,flight_id,duration,total_passengers,total_seats,fuel_expence,tickets_total_cost,income,price_per_km
count,193.000000,193.000000,193.000000,193.000000,193.000000,1.800000e+02,1.800000e+02,180.000000
mean,136540.466321,90.538860,95.082902,114.611399,76098.228411,1.179647e+06,1.099657e+06,1146.113304
std,249.389171,62.864684,29.159643,16.505342,17267.354429,4.811836e+05,4.765547e+05,171.577073
min,136119.000000,49.000000,0.000000,97.000000,4249.000000,5.310000e+05,4.558458e+05,723.725006
25%,136320.000000,50.000000,90.000000,97.000000,75154.166667,7.385250e+05,6.633708e+05,1053.015401
50%,136534.000000,99.000000,97.000000,130.000000,76657.250000,1.068850e+06,9.895840e+05,1095.712104
75%,136768.000000,100.000000,110.000000,130.000000,84881.000000,1.631950e+06,1.548660e+06,1269.526260
max,136961.000000,312.000000,130.000000,130.000000,125202.000000,1.886000e+06,1.805363e+06,1479.960093


### В нашем наборе есть 13 рейсов в город Новокузнецк, которые не состоялись -- удалим их из набора.

In [4]:
data.dropna(axis=0, subset=['income'], inplace=True)

## Начнём по разным признакам собирать "худшие" рейсы за зиму.

Сделаем список десяти худших рейсов по показателю "price_per_km" (отношение прибыли к кол-ву пролетевших километров).

In [5]:
list0 = list(data.sort_values(by='price_per_km').head(10).flight_id)

### Добавим колонку с процентом занятых мест в самолёте.

In [6]:
data['occupation_rate'] = data.total_passengers / data.total_seats

### Добавим колонку с отношением прибыли к кол-ву пассажиров.

In [7]:
data['income_per_passenger'] = data.income / data.total_passengers

Сделаем список десяти худших рейсов по показателю "income_per_passenger".

In [8]:
list1 = list(data[['flight_id', 'income_per_passenger', 'departure_date']].sort_values(by='income_per_passenger').head(10).flight_id)

### Добавим колонку с отношением затрат на топливо к общей выручке.

Чем больше -- тем, соответственно, хуже.

In [9]:
data['income_rate'] = data.fuel_expence / data.tickets_total_cost

Сделаем список десяти худших рейсов по показателю "income_rate".

In [10]:
list2 = list(data[['flight_id', 'income_rate', 'departure_date']].sort_values(by='income_rate', ascending=False).head(10).flight_id)

Добавим колонку с днем недели для анализа полученных результатов (1 - Понедельник, ... , 7 - Воскресенье)

In [11]:
data['day_of_week'] = data.departure_date.apply(lambda x: datetime.date.fromisoformat(x).isoweekday())

### Теперь посмотрим, какие рейсы появляются во всех трёх списках.

In [12]:
cancel = set(list0).intersection(set(list1)).intersection(set(list2))

In [13]:
data.loc[data.flight_id.isin(cancel)]

,flight_id,origin,destination,duration,aircraft_model,total_passengers,total_seats,fuel_expence,tickets_total_cost,income,departure_date,price_per_km,occupation_rate,income_per_passenger,income_rate,day_of_week
14,136807,Anapa,Belgorod,50,Sukhoi Superjet-100,68,97,75154.166667,531000.0,455845.833333,2017-02-23,723.725006,0.701031,6703.615196,0.141533,4
16,136887,Anapa,Belgorod,49,Sukhoi Superjet-100,78,97,73651.083333,595200.0,521548.916667,2017-01-20,828.038703,0.804124,6686.524573,0.123742,5
19,136844,Anapa,Belgorod,50,Sukhoi Superjet-100,79,97,75154.166667,575100.0,499945.833333,2017-02-28,793.740503,0.814433,6328.428270,0.130680,2
23,136937,Anapa,Belgorod,50,Sukhoi Superjet-100,81,97,75154.166667,626100.0,550945.833333,2017-01-03,874.710806,0.835052,6801.800412,0.120035,2


In [15]:
data.income.mean()

1099656.6550925928

# Вывод

Проведя анализ рейсов из Анапы за зиму 2016-2017 года я могу выделить 3 рейса, которые показали себя с наименее выгодной стороны. ID этих рейсов ```136807, 136887, 136844```.

Эти рейсы получились наименее выгодные по трём критериям: 
1.	price_per_km" (отношение прибыли к кол-ву пролетевших километров)
2.	income_per_passenger (доход в пересчёте на голову пассажира)
3.	income_rate (показатель затрат на топливо к общей выручке)

Можно также заметить, что один из этих рейсов приходится на праздник 23 февраля, другой – это последние день месяца февраля (канун 1-го марта). А дату 3-го рейса, 20 января, можно объяснить как первая пятница после праздников, когда все уже давно вернулись домой и погрузились в работу.

Хочется подчеркнуть, что доходность этих рейсов 455846 руб., 521549 руб., 499946 руб. соответственно, при средней доходности всех рейсов в 1 099 657 рублей.